<a href="https://colab.research.google.com/github/jai2shan/TSAI-EVA40-Assignments/blob/master/Assignment%20S6%20-%20Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.listdir()

['.config', 'asgnmnt6-0.0.32-py3-none-any.whl', 'sample_data']

In [2]:
pip install asgnmnt6-0.0.32-py3-none-any.whl

Processing ./asgnmnt6-0.0.32-py3-none-any.whl


## Loading Packages

In [0]:
from __future__ import print_function
import torch
import torch.optim as optim


import os

from GBNModel import GBNNet
from Model import Net
from DataTransformation import DataTransformation
from TrainTest import TrainTest